<img src="HotColdAnalysisDraft.jpg" alt="HotColdAnalysisDraft" style="zoom: 25%;" />

In [7]:
# original contour size 6
# we can the original contour by using canny and findContours(external)
OCNT = ['CNT0', 'CNT1', 'CNT2', 'CNT3','CNT4', 'CNT5']

# RR size 9
# we can calculate RR by radius, ex raidus = 300, img(900,900)
# drwa 9 rectangle(square), as a 3*3 square
RR = ['RR0', 'RR1', 'RR2', 'RR3', 'RR4', 'RR5', 'RR6', 'RR7', 'RR8', 'RR9']

# group Chunked contour by RR
# using RR square a sliding window, so using the RR window to get chunked contours
# by canny and findContours(external)
RRCC = [['RR0-CC0','RR0-CC1','RR0-CC2'],\
      ['RR1-CC0','RR0-CC1','RR0-CC2'],\
      ['RR2-CC0'],\
      ['RR3-CC0','RR0-CC1'],\
      ['RR4-CC0','RR0-CC1'],\
      ['RR5-CC0'],\
      ['RR6-CC0'],\
      ['RR7-CC0'],\
      ['RR8-CC0'],\
       ]

# group contour by RR
# we may not use this, since we can use contour's bounding rectanle's 4 points
# to get the related RR region,since we have the RR region's points range
RRCNT = [['RR0-CNT0','RR0-CNT3','RR0-CNT4'],\
      ['RR1-CNT1','RR1-CNT2','RR1-CNT4'],\
      ['RR2-CNT2'],\
      ['RR3-CNT3','RR3-CNT4'],\
      ['RR4-CNT4','RR4-CNT5'],\
      ['RR5-CNT5'],\
      ['RR6-CNT3'],\
      ['RR7-CNT5'],\
      ['RR8-CNT5'],\
       ]

# group Chunked contour by original contour
# orginal contour should include chunked contour
# if a chunedContour could intersect with a original contour, 
# then the chunkedContour belongs to the original contour
# intersectConvexConvex  find the intersection
# so basically we don't need area for this, only need the relationship of belonging
# maybe useful when combine 2 contours in the same HotColdRegion
CNTCC = [['CNT0-RR0-CC0'],\
       ['CNT1-RR1-CC0'],\
       ['CNT2-RR1-CC1', 'CNT2-RR2-CC0'],\
       ['CNT3-RR0-CC2', 'CNT3-RR3-CC0', 'CNT3-RR6-CC0'],\
       ['CNT4-RR0-CC1', 'CNT4-RR1-CC2', 'CNT4-RR3-CC1', 'CNT4-RR4-CC0'],\
       ['CNT5-RR4-CC1', 'CNT5-RR5-CC0', 'CNT5-RR7-CC0', 'CNT5-RR8-CC0'],\
      ]

# HotColdRegion is the result based on above data
# group Chunked contour by HotColdRegion
# The biggest Chunked contour in the RR is used to mark the HoldCodeRegion
# ccArea - chunked contour area, ccArea = countourArea(chunkedContour)
# rrArea - RR square, rrArea = RR_Radius * RR_Radius
# ccrrRatio - ccArea/rrArea
#
# if the ratio is 100% and have more than 1 HC region, the top and left HC is the root HC
# if 2 contour in the same HC, combine them.

HCCC =  [['RR0-CC0-70','RR0-CC1-10','RR0-CC2-10'],\
      ['RR1-CC0-30','RR0-CC1-10','RR0-CC2-40'],\
      ['RR2-CC0-60'],\
      ['RR3-CC0-50','RR0-CC1-20'],\
      ['RR4-CC0-45','RR0-CC1-40'],\
      ['RR5-CC0-10'],\
      ['RR6-CC0-10'],\
      ['RR7-CC0-20'],\
      ['RR8-CC0-5'],\
       ]

# After compare ccrrRatio and find the biggest ccrrRatio, we get the HC
# HC could have multiple chunked contour in one, we can use it to combine contour later
# if there are 2 100 from same contour, then we use the first one
# so basically the chunked contour with original contour's relateship is necessary
HCCCResult = [['RR0-CC0-70'],\
       ['RR1-CC0-30'],\
       ['RR2-CC0-60'],\
       ['RR3-CC0-50'],\
       ['RR4-CC0-45','RR0-CC1-40'],\
      ]

In [5]:
class ChunkedContour:
    
    'chunked contour index in the all the chunked contours, access chunked contour by index'
    ccIndex = -1
    
    'chunked contour original contour'
    ocIndex = -1
    
    'RR region index in N*N the matrix, N is the number RR region in a Row'
    rrIndex = -1
    
    """
    ccArea - chunked contour area, ccArea = countourArea(chunkedContour)
    rrArea - RR square, rrArea = RR_Radius * RR_Radius
    ccrrRatio - ccArea/rrArea
    """
    ccrrRatio = 0
    
    
    def __init__(self, ccIndex, ocIndex, rrIndex, ccrrRatio):
        self.ccIndex = ccIndex
        self.ocIndex = ocIndex
        self.rrIndex = rrIndex
        self.ccrrRatio = ccrrRatio
        
    def __str__(self):
        return 'ChunkedContour (CC %d, OC %d, RR %d, Ratio %d)' %(self.ccIndex, self.ocIndex, self.rrIndex, self.ccrrRatio)

    
HCMat =  [[70.12, 10, 10],\
      [30,10,40],\
      [60],\
      [50,20],\
      [45,40],\
      [10],\
      [10],\
      [20],\
      [5],\
       ]

ccMat = []
for rrIndex,rr in enumerate(HCMat):
    for ccIndex,ccArea in enumerate(rr):
        cc = ChunkedContour(ccIndex, -1, rrIndex, ccArea)
        ccMat.append(cc)
        
for cc in ccMat:  
     print(cc)


ChunkedContour (CC 0, OC -1, RR 0, Ratio 70)
ChunkedContour (CC 1, OC -1, RR 0, Ratio 10)
ChunkedContour (CC 2, OC -1, RR 0, Ratio 10)
ChunkedContour (CC 0, OC -1, RR 1, Ratio 30)
ChunkedContour (CC 1, OC -1, RR 1, Ratio 10)
ChunkedContour (CC 2, OC -1, RR 1, Ratio 40)
ChunkedContour (CC 0, OC -1, RR 2, Ratio 60)
ChunkedContour (CC 0, OC -1, RR 3, Ratio 50)
ChunkedContour (CC 1, OC -1, RR 3, Ratio 20)
ChunkedContour (CC 0, OC -1, RR 4, Ratio 45)
ChunkedContour (CC 1, OC -1, RR 4, Ratio 40)
ChunkedContour (CC 0, OC -1, RR 5, Ratio 10)
ChunkedContour (CC 0, OC -1, RR 6, Ratio 10)
ChunkedContour (CC 0, OC -1, RR 7, Ratio 20)
ChunkedContour (CC 0, OC -1, RR 8, Ratio 5)
